In [1]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
# from box_packing.models import Box, Item, db
from boxes import default_boxes

In [2]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite+pysqlite:///moving.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

In [3]:
class Box(db.Model):
    __tablename__ = 'boxes'
    id = db.Column('id', db.Integer, primary_key=True)
    name = db.Column('name', db.String(30))
    size = db.Column('size', db.String(20))
    length = db.Column('length', db.Integer)
    width = db.Column('width', db.Integer)
    height = db.Column('height', db.Integer)
    weight_class = db.Column('weight_class', db.Integer)
    fragile = db.Column('fragile', db.Boolean, default=False)
    liquid = db.Column('liquid', db.Boolean, default=False)
    valuable = db.Column('valuable', db.Boolean, default=False)
    weather_resistant = db.Column('weather_resistant', db.Boolean, default=False)

    items = db.relationship('Item', backref='box')

    # def __repr__(self):
    #     return f'Box(id={self.id!r}, name={self.name!r}, size={self.size!r}'


class Item(db.Model):
    __tablename__ = 'items'

    id = db.Column('id', db.Integer, primary_key=True)
    box_id = db.Column('box_id', db.ForeignKey('boxes.id'), nullable=False)
    name = db.Column('name', db.String)

    # boxes = db.relationship('Box', backref='items')

    def __repr__(self):
        return f'Item(id={self.id!r}, name={self.name!r}, box_id={self.box_id!r}'


db.create_all()

In [4]:
# engine = create_engine('sqlite+pysqlite:///moving.db', echo=False, future=True)
# session = Session(engine)

In [20]:
def add_box(box):
    db.session.add(Box(**box))
    db.session.commit()


def add_default_boxes():
    box_objects = [Box(**box) for box in default_boxes]
    db.session.add_all(box_objects)
    db.session.commit()


def delete_box(id):
    db.session.query(Box).filter(Box.id == id).delete()
    db.session.commit()


def delete_all_boxes():
    db.session.query(Box).delete()
    db.session.commit()


def get_box(id):
    box = db.session.query(Box).filter(Box.id == id).one_or_none()
    if box is None:
        return 'No box with that id'
    border_size = 30
    box_attrs = [a for a in vars(box) if not a.startswith('__') and not a.startswith('_')]
    title = f'{"="*border_size} BOX {box.id} {"="*border_size}'
    print(title)
    for item in box_attrs:
        print(f'||    {item}: {getattr(box, item)}')
    print('-' * len(title))
    print()


def print_all_boxes():
    boxes = db.session.query(Box).all()
    border_size = 30
    for box in sorted(boxes, key=lambda x: x.id):
        box_attrs = [
            a for a in vars(box) if not a.startswith('__') and not a.startswith('_')
        ]
        title = f'{"="*border_size} BOX {box.id} {"="*border_size}'
        print(title)
        for item in box_attrs:
            print(f'||    {item}: {getattr(box, item)}')
        print('-' * len(title))
        print()


def update_box(id, **kwargs):
    box = db.session.query(Box).filter(Box.id == id).first()
    print(vars(box))
    for k, v in kwargs.items():
        setattr(box, k, v)
    print(vars(box))
    db.session.commit()


def add_box_item(box_id: str, item: str) -> None:
    item_object = Item(**{'box_id': box_id, 'name': item})
    db.session.add(item_object)
    db.session.commit()


def delete_box_item(box_id: str, item: str) -> None:
    db.session.query(Item).filter(Item.box_id==box_id, Item.name==item).delete()
    db.session.commit()

def print_box_items(box_id):
    items = db.session.query(Box).filter(Box.id==box_id).first().items
    for item in items:
        print(item.name)

In [18]:
# delete_all_boxes()
add_default_boxes()

In [21]:
print_box_items(1)

    

power cable
cords
chargers
airpods
fans


In [8]:
add_box_item(1, 'new computer')

In [13]:
db.session.query(Box).filter(Box.id==1).first().items

[Item(id=1, name='power cable', box_id=1,
 Item(id=2, name='cords', box_id=1,
 Item(id=3, name='chargers', box_id=1,
 Item(id=4, name='airpods', box_id=1,
 Item(id=5, name='fans', box_id=1]

In [14]:
delete_box_item(1, 'new computer')